In [1]:
import pandas as pd
df=pd.read_csv("fertility_diagnosis.csv",header=None)
df

,0,1,2,3,4,5,6,7,8,9
0,-0.33,0.69,0,1,1,0,0.8,0,0.88,N
1,-0.33,0.94,1,0,1,0,0.8,1,0.31,O
2,-0.33,0.50,1,0,0,0,1.0,-1,0.50,N
3,-0.33,0.75,0,1,1,0,1.0,-1,0.38,N
4,-0.33,0.67,1,1,0,0,0.8,-1,0.50,O
...,...,...,...,...,...,...,...,...,...,...
95,-1.00,0.67,1,0,0,0,1.0,-1,0.50,N
96,-1.00,0.61,1,0,0,0,0.8,0,0.50,N
97,-1.00,0.67,1,1,1,0,1.0,-1,0.31,N
98,-1.00,0.64,1,0,1,0,1.0,0,0.19,N


In [2]:
X=df.iloc[:,0:9]
X.head()

,0,1,2,3,4,5,6,7,8
0,-0.33,0.69,0,1,1,0,0.8,0,0.88
1,-0.33,0.94,1,0,1,0,0.8,1,0.31
2,-0.33,0.50,1,0,0,0,1.0,-1,0.50
3,-0.33,0.75,0,1,1,0,1.0,-1,0.38
4,-0.33,0.67,1,1,0,0,0.8,-1,0.50


In [3]:
y=pd.DataFrame(df.iloc[:,9])
y.head()

,9
0,N
1,O
2,N
3,N
4,O


In [4]:
from sklearn import preprocessing
scaler=preprocessing.StandardScaler()
X=scaler.fit_transform(X)
X=pd.DataFrame(X)
X

,0,1,2,3,4,5,6,7,8
0,-0.316753,0.173970,-2.586949,1.128152,0.980196,-0.32881,-0.192006,0.434959,2.551481
1,-0.316753,2.245043,0.386556,-0.886405,0.980196,-0.32881,-0.192006,1.677698,-0.521943
2,-0.316753,-1.400045,0.386556,-0.886405,-1.020204,-0.32881,1.008032,-0.807781,0.502532
3,-0.316753,0.671028,-2.586949,1.128152,0.980196,-0.32881,1.008032,-0.807781,-0.144505
4,-0.316753,0.008284,0.386556,1.128152,-1.020204,-0.32881,-0.192006,-0.807781,0.502532
...,...,...,...,...,...,...,...,...,...
95,-1.161931,0.008284,0.386556,-0.886405,-1.020204,-0.32881,1.008032,-0.807781,0.502532
96,-1.161931,-0.488773,0.386556,-0.886405,-1.020204,-0.32881,-0.192006,0.434959,0.502532
97,-1.161931,0.008284,0.386556,1.128152,0.980196,-0.32881,1.008032,-0.807781,-0.521943
98,-1.161931,-0.240244,0.386556,-0.886405,0.980196,-0.32881,1.008032,0.434959,-1.168979


In [5]:
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif

# feature extraction. We are going to output the selection scores for all features and select the features with the highest scores.
test = SelectKBest(score_func=mutual_info_classif, k="all")
fit = test.fit(X, y)
features = fit.transform(X)
# summarize scores
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


,scores
3,0.064153
2,0.024888
1,0.011783
0,0.000000
4,0.000000
5,0.000000
6,0.000000
7,0.000000
8,0.000000


In [6]:
features=scores[scores["scores"]>0]
features

,scores
3,0.064153
2,0.024888
1,0.011783


In [7]:
X=X.iloc[:,features.index]
X.head()

,3,2,1
0,1.128152,-2.586949,0.173970
1,-0.886405,0.386556,2.245043
2,-0.886405,0.386556,-1.400045
3,1.128152,-2.586949,0.671028
4,1.128152,0.386556,0.008284


In [8]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
y=pd.DataFrame(enc.fit_transform(y).toarray())
y.head()

,0,1
0,1.0,0.0
1,0.0,1.0
2,1.0,0.0
3,1.0,0.0
4,0.0,1.0


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [10]:
length=X_train.shape[1]
num_classes=y_test.shape[1]

In [11]:
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

Optimize the batch size and the number of epochs.

In [14]:

def create_model():
    model = Sequential()
    model.add(Dense(12, input_dim=length, activation='relu'))
    model.add(Dense(num_classes, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

seed = 7
numpy.random.seed(seed)
model = KerasClassifier(build_fn=create_model,verbose=0)
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.870279 using {'batch_size': 20, 'epochs': 100}
0.770525 (0.029404) with: {'batch_size': 10, 'epochs': 10}
0.870279 (0.018578) with: {'batch_size': 10, 'epochs': 50}
0.870279 (0.018578) with: {'batch_size': 10, 'epochs': 100}
0.530788 (0.101219) with: {'batch_size': 20, 'epochs': 10}
0.818145 (0.029898) with: {'batch_size': 20, 'epochs': 50}
0.870279 (0.018578) with: {'batch_size': 20, 'epochs': 100}
0.629310 (0.160499) with: {'batch_size': 40, 'epochs': 10}
0.823686 (0.011922) with: {'batch_size': 40, 'epochs': 50}
0.870279 (0.018578) with: {'batch_size': 40, 'epochs': 100}
0.646757 (0.068073) with: {'batch_size': 60, 'epochs': 10}
0.722291 (0.077601) with: {'batch_size': 60, 'epochs': 50}
0.870279 (0.018578) with: {'batch_size': 60, 'epochs': 100}
0.507389 (0.100575) with: {'batch_size': 80, 'epochs': 10}
0.789409 (0.072028) with: {'batch_size': 80, 'epochs': 50}
0.818555 (0.054571) with: {'batch_size': 80, 'epochs': 100}
0.590107 (0.112717) with: {'batch_size': 100, 'epochs':

The optimal batch size is 20 and the optimal number of epochs is 100. Implement the results and optimize other hyperparameters.

Search for the best optimizer.

In [13]:
def create_model(optimizer='adam'):
    model = Sequential()
    model.add(Dense(12, input_dim=length, activation='relu'))
    model.add(Dense(num_classes, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

seed = 7
numpy.random.seed(seed)
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=20, verbose=0)
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best: 0.870588 using {'optimizer': 'RMSprop'}
0.847059 (0.018076) with: {'optimizer': 'SGD'}
0.870588 (0.018684) with: {'optimizer': 'RMSprop'}
0.870588 (0.018684) with: {'optimizer': 'Adagrad'}
0.870588 (0.031925) with: {'optimizer': 'Adadelta'}
0.870588 (0.018684) with: {'optimizer': 'Adam'}
0.858824 (0.030777) with: {'optimizer': 'Adamax'}
0.870588 (0.018684) with: {'optimizer': 'Nadam'}


The algorithm chose "RMSprop" as the optimal optimizer. Indeed, a closer look at the output shows that "SGD", "Adagrad","RMSProp","Adam" or "Nadam" would make no difference. The only two optimizers that would make a difference is "Adadelta" and "Adamax".

Implement the best optimizer and search for the optimal learning rate.

In [14]:
from keras.optimizers import RMSprop

def create_model(learn_rate=0.01, momentum=0):
    model = Sequential()
    model.add(Dense(12, input_dim=length, activation='relu'))
    model.add(Dense(num_classes, activation='sigmoid'))
    optimizer = RMSprop(lr=learn_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

seed = 7
numpy.random.seed(seed)
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=20, verbose=0)
learn_rate = [0.001, 0.005,0.01, 0.05,0.1, 0.2, 0.3]
param_grid = dict(learn_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best: 0.870588 using {'learn_rate': 0.001}
0.870588 (0.018684) with: {'learn_rate': 0.001}
0.864706 (0.030835) with: {'learn_rate': 0.005}
0.858824 (0.026269) with: {'learn_rate': 0.01}
0.870588 (0.034488) with: {'learn_rate': 0.05}
0.870588 (0.015738) with: {'learn_rate': 0.1}
0.811765 (0.018431) with: {'learn_rate': 0.2}
0.847059 (0.014013) with: {'learn_rate': 0.3}


The optimal learning rate is 0.001.

Implement the optimial learning rate and search for the optimal initialization for the neural network.

In [15]:
def create_model(init_mode='uniform'):
    model = Sequential()
    model.add(Dense(12, input_dim=length, kernel_initializer=init_mode, activation='relu'))
    model.add(Dense(num_classes, kernel_initializer=init_mode, activation='sigmoid'))
    opt= RMSprop(lr=0.001)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

seed = 7
numpy.random.seed(seed)
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=20, verbose=0)
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best: 0.870588 using {'init_mode': 'uniform'}
0.870588 (0.018684) with: {'init_mode': 'uniform'}
0.870588 (0.018684) with: {'init_mode': 'lecun_uniform'}
0.870588 (0.018684) with: {'init_mode': 'normal'}
0.870588 (0.018684) with: {'init_mode': 'zero'}
0.870588 (0.018684) with: {'init_mode': 'glorot_normal'}
0.870588 (0.018684) with: {'init_mode': 'glorot_uniform'}
0.870588 (0.018684) with: {'init_mode': 'he_normal'}
0.852941 (0.038178) with: {'init_mode': 'he_uniform'}


The optimal initialization is "uniform".

Implement the optimal initialization and search for the optimal neuron activation function.

In [16]:
def create_model(activation='relu'):
    model = Sequential()
    model.add(Dense(12, input_dim=length, kernel_initializer='uniform', activation=activation))
    model.add(Dense(num_classes, kernel_initializer='uniform', activation='sigmoid'))
    opt= RMSprop(lr=0.001)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

seed = 7
numpy.random.seed(seed)
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=20, verbose=0)
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best: 0.870588 using {'activation': 'softmax'}
0.870588 (0.018684) with: {'activation': 'softmax'}
0.870588 (0.018684) with: {'activation': 'softplus'}
0.858824 (0.030777) with: {'activation': 'softsign'}
0.870588 (0.018684) with: {'activation': 'relu'}
0.858824 (0.030777) with: {'activation': 'tanh'}
0.870588 (0.018684) with: {'activation': 'sigmoid'}
0.870588 (0.018684) with: {'activation': 'hard_sigmoid'}
0.858824 (0.030777) with: {'activation': 'linear'}


The optimal activation function is "softmax".

Implement "softmax" and search for the optimal dropout rate.

In [17]:
from keras.constraints import maxnorm
from keras.layers import Dropout
dim=X_train.shape[1]
def create_model(dropout_rate=0.0, weight_constraint=0):
    model = Sequential()
    model.add(Dense(12, input_dim=length, kernel_initializer='uniform', activation='softmax', kernel_constraint=maxnorm(weight_constraint)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, kernel_initializer='uniform', activation='softmax'))
    opt= RMSprop(lr=0.001)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

seed = 7
numpy.random.seed(seed)
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=20, verbose=0)
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best: 0.870588 using {'dropout_rate': 0.0, 'weight_constraint': 1}
0.870588 (0.018684) with: {'dropout_rate': 0.0, 'weight_constraint': 1}
0.870588 (0.018684) with: {'dropout_rate': 0.0, 'weight_constraint': 2}
0.870588 (0.018684) with: {'dropout_rate': 0.0, 'weight_constraint': 3}
0.870588 (0.018684) with: {'dropout_rate': 0.0, 'weight_constraint': 4}
0.870588 (0.018684) with: {'dropout_rate': 0.0, 'weight_constraint': 5}
0.870588 (0.018684) with: {'dropout_rate': 0.1, 'weight_constraint': 1}
0.870588 (0.018684) with: {'dropout_rate': 0.1, 'weight_constraint': 2}
0.870588 (0.018684) with: {'dropout_rate': 0.1, 'weight_constraint': 3}
0.870588 (0.018684) with: {'dropout_rate': 0.1, 'weight_constraint': 4}
0.870588 (0.018684) with: {'dropout_rate': 0.1, 'weight_constraint': 5}
0.870588 (0.018684) with: {'dropout_rate': 0.2, 'weight_constraint': 1}
0.870588 (0.018684) with: {'dropout_rate': 0.2, 'weight_constraint': 2}
0.870588 (0.018684) with: {'dropout_rate': 0.2, 'weight_constraint': 

Optimal dropout rate is 0 and optimal weight_constraint is 1.

Implement the optimal dropout rate, weight-constraint and search for the optimal number of neurons for the hidden layers.

In [18]:
from keras.constraints import maxnorm
from keras.layers import Dropout
from keras.optimizers import RMSprop
import numpy as np
def create_model(neurons=1):
    model = Sequential()
    model.add(Dense(12, input_dim=length, kernel_initializer='uniform', activation='softmax', kernel_constraint=maxnorm(1)))
    model.add(Dropout(0))
    model.add(Dense(num_classes, kernel_initializer='uniform', activation='softmax'))
    opt= RMSprop(lr=0.001)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model
seed = 7
numpy.random.seed(seed)
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=20, verbose=0)
neurons = np.arange(10,100,5)
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best: 0.870588 using {'neurons': 10}
0.870588 (0.018684) with: {'neurons': 10}
0.870588 (0.018684) with: {'neurons': 15}
0.870588 (0.018684) with: {'neurons': 20}
0.870588 (0.018684) with: {'neurons': 25}
0.870588 (0.018684) with: {'neurons': 30}
0.870588 (0.018684) with: {'neurons': 35}
0.870588 (0.018684) with: {'neurons': 40}
0.870588 (0.018684) with: {'neurons': 45}
0.870588 (0.018684) with: {'neurons': 50}
0.870588 (0.018684) with: {'neurons': 55}
0.870588 (0.018684) with: {'neurons': 60}
0.870588 (0.018684) with: {'neurons': 65}
0.870588 (0.018684) with: {'neurons': 70}
0.870588 (0.018684) with: {'neurons': 75}
0.870588 (0.018684) with: {'neurons': 80}
0.870588 (0.018684) with: {'neurons': 85}
0.870588 (0.018684) with: {'neurons': 90}
0.870588 (0.018684) with: {'neurons': 95}


The optimal number of neurons for hidden layers is 10. Indeed, the results show that it makes no difference whether we put 10 of 95 neurons on the hidden layers. For efficiency, we will go with 10.

Here is a summary of the optimization results: </br>
- Optimal batch size: 20
- Optimal number of epochs: 100
- Optimal optimization algorithm: SGD", "Adagrad","RMSProp","Adam" or "Nadam".
- OPtimal learning rate: 0.001
- Optimal initializer: uniform
- Optimal activation function: softmax
- Optimal dropout rate: 0
- Optimal weight constraint: 1
- Optimal number of neurons for hidden layers: 10

The Keras neural network model based on the optimization results:

In [37]:
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.constraints import maxnorm
from keras.optimizers import Adam
#from keras.optimizers import Adam

def classification_model():
    model = Sequential()
    model.add(Dense(10, input_dim=length, kernel_initializer='uniform', activation='softmax', kernel_constraint=maxnorm(1)))
    model.add(Dense(10,activation='softmax'))
    model.add(Dense(10,activation='softmax'))
    model.add(Dense(10,activation='softmax'))
    model.add(Dense(num_classes, kernel_initializer='uniform', activation='softmax'))
    opt= Adam(lr=0.001)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

#model = KerasClassifier(build_fn=classification_model, epochs=100, batch_size=20, verbose=0)

In [15]:
model=classification_model()
model.fit(X_train,y_train,validation_data=(X_test, y_test),epochs=100, batch_size=20,verbose=2)
model.evaluate(X_test,y_test,verbose=0)

Train on 85 samples, validate on 15 samples
Epoch 1/100
 - 12s - loss: 0.8730 - acc: 0.3706 - val_loss: 0.8203 - val_acc: 0.4667
Epoch 2/100
 - 0s - loss: 0.8552 - acc: 0.3765 - val_loss: 0.8057 - val_acc: 0.4667
Epoch 3/100
 - 0s - loss: 0.8388 - acc: 0.3824 - val_loss: 0.7918 - val_acc: 0.4667
Epoch 4/100
 - 0s - loss: 0.8212 - acc: 0.4000 - val_loss: 0.7782 - val_acc: 0.4667
Epoch 5/100
 - 0s - loss: 0.8051 - acc: 0.4118 - val_loss: 0.7649 - val_acc: 0.4667
Epoch 6/100
 - 0s - loss: 0.7897 - acc: 0.4176 - val_loss: 0.7521 - val_acc: 0.5667
Epoch 7/100
 - 0s - loss: 0.7748 - acc: 0.4706 - val_loss: 0.7397 - val_acc: 0.6000
Epoch 8/100
 - 0s - loss: 0.7602 - acc: 0.5059 - val_loss: 0.7278 - val_acc: 0.6333
Epoch 9/100
 - 0s - loss: 0.7464 - acc: 0.5176 - val_loss: 0.7164 - val_acc: 0.5667
Epoch 10/100
 - 0s - loss: 0.7329 - acc: 0.5235 - val_loss: 0.7056 - val_acc: 0.5667
Epoch 11/100
 - 0s - loss: 0.7213 - acc: 0.5412 - val_loss: 0.6949 - val_acc: 0.6000
Epoch 12/100
 - 0s - loss: 0.

[0.36658674478530884, 0.9333333373069763]

In [16]:
y_pred=model.predict(X_test)
y_pred

array([[0.722, 0.284],
       [0.844, 0.121],
       [0.888, 0.079],
       [0.876, 0.06 ],
       [0.873, 0.093],
       [0.72 , 0.167],
       [0.795, 0.187],
       [0.699, 0.32 ],
       [0.795, 0.187],
       [0.663, 0.36 ],
       [0.765, 0.227],
       [0.81 , 0.099],
       [0.894, 0.05 ],
       [0.844, 0.121],
       [0.74 , 0.255]], dtype=float32)

The model predicted all semens in the test data to be "Normal (N)". However, there is 1 instance of "Altered (O)" semen in the test data, row index 70, as shown below. This is the 3rd instance in the test data. However, the model predicted that there was only 6% (0.06) chance that the semen would be "Altered (O)". 

In [44]:
y_test

,0,1
83,1.0,0.0
53,1.0,0.0
70,0.0,1.0
45,1.0,0.0
44,1.0,0.0
39,1.0,0.0
22,1.0,0.0
80,1.0,0.0
10,1.0,0.0
0,1.0,0.0


Even with all the hyper-parameters fined-tune, even with a 93.33% accuracy on test data, if this model is to be used for screening for semens that are "altered", labeled as "O" in the dataset, it is practically useless. [It is my own work, but that does not mean I won't criticize it. :) ]

This is the end of this experiment. It illustrates (1) the limitation of fine-tuning hyper-parameters; (2) the problem with an imbalanced dataset.

For dataset with uneven class distribution, it seems to me that the most effective and the only way to deal with with it is to recalibrate the class distribution with synthetic data. 